# CC Recon 
![transform](Banner-01.png)

This jupyter notebook will read Coles Credit Card, and extract data out of it :









In [70]:
# Import requried packages
from datetime import datetime
from datetime import date
from pathlib import Path
import fitz  # PyMuPDF
import json
import re


In [71]:
# Static data and file path.

#file_path = "C:/Users/maddy/Dropbox/DeletedToBe/MonthlyExpenseEstimation/Coles-May2025.pdf"
#file_path = "C:/Users/maddy/Dropbox/DeletedToBe/MonthlyExpenseEstimation/Coles-Jun2025.pdf"
file_path = "C:/Users/maddy/Dropbox/DeletedToBe/MonthlyExpenseEstimation/Coles-Jul2025.pdf"



# Map short month names to numbers
month_order = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}


# Static References 
payment_description = "Bpay Payments"

# Regex patterns
date_pattern = re.compile(r"^[A-Z][a-z]{2} \d{2}$")  # Matches 'Mon DD'
ref_pattern = re.compile(r"^(BMR.{15}|[0-9]{23})$")  # 18-char starting with BMR
payment_ref_pattern = re.compile(r"^BMR.{15}$")  # 18-char starting with BMR OR 23-digit number
amount_pattern = re.compile(r"^-?\d+\.\d{2}$")       # Amount with 2 decimal places, may be negative
negative_amount_pattern = re.compile(r"^-\d+\.\d{2}$")       # Negative Amount with 2 decimal places.

# Final result lines
tsv_lines = []


# Assume current year
current_year = date.today().year

# Final result lines with date tag
tsv_with_dates = []


In [72]:
# Load PDF and extract text
pdf_path = Path(file_path)
doc = fitz.open(pdf_path)
all_text = "\n".join([page.get_text() for page in doc])
#print(all_text)
doc.close()

In [73]:
### Read PDF text in order.
### For debugging purpose Print in JSON


# Load PDF
pdf_path = Path(file_path)
doc = fitz.open(pdf_path)

# Collect text blocks in order
all_lines = []
for page in doc:
    blocks = page.get_text("blocks")  # returns list of (x0, y0, x1, y1, text, block_no, block_type)
    blocks.sort(key=lambda b: (b[1], b[0]))  # sort by y (top-down), then x (left-right)
    for b in blocks:
        lines = b[4].splitlines()  # split block text into lines
        all_lines.extend([line.strip() for line in lines if line.strip()])

doc.close()

# Debug 
# Convert to JSON array
json_output = json.dumps(all_lines, indent=2, ensure_ascii=False)
print(json_output)


[
  "E-9528  S-12961  I-25921",
  "Page 1 of 4, July 2025",
  "Customer Support",
  "1300 306 397",
  "Web",
  "coles.com.au/mastercard",
  "Account Number",
  "5240 1467 5024 1143",
  "Flybuys Membership Number",
  "6008 9482 3671 5719",
  "Mradulanand Sharma",
  "11 Els Ct",
  "BERWICK VIC 3806",
  "Statement Period",
  "12/06/25 - 13/07/25",
  "Overlimit Amount (Pay immediately)",
  "$0.00",
  "Overdue Amount (Pay immediately)",
  "$0.00",
  "Minimum Payment Due",
  "$119.00",
  "Payment Due Date",
  "28 July 2025",
  "Coles No Annual Fee Platinum Mastercard",
  "Account Summary",
  "Credit Limit",
  "Opening",
  "Balance",
  "Instalment",
  "Balance",
  "(not yet due)",
  "Closing",
  "Balance^",
  "Closing Balance",
  "less Instalment",
  "Balance^^",
  "Available",
  "Credit*",
  "Pay this amount",
  "by the due date",
  "to maintain",
  "interest free days",
  "$10,000.00",
  "$9,797.82",
  "$0.00",
  "$5,945.46",
  "$5,945.46",
  "$4,055.00",
  "Total Cash Limit: $10,000.00 of 

In [74]:
### Find closing balance


# Initialize variable to store closing balance
closing_balance = None

# Start from the end of the list and iterate backward
for i in range(len(all_lines) - 1, 0, -1):
    current_line = all_lines[i].strip()
    previous_line = all_lines[i - 1].strip()

    # Debug output
    # print(f"Checking lines at index {i-1} and {i}: '{previous_line}' -> '{current_line}'")

    # Check for pattern: previous line is "Closing Balance", current line starts with $
    if previous_line == "Closing Balance" and current_line.startswith("$"):
        closing_balance = current_line
        break  # Stop at first match found

# Print result
if closing_balance:
    print(f"Closing Balance: {closing_balance}")
else:
    print("No closing balance found.")


Closing Balance: $5,945.46


In [75]:
### Due Date from credit card

from datetime import datetime

# Initialize due date variable
due_date = None

# Iterate through all_lines to find "Payment Due Date"
for i in range(len(all_lines) - 1):
    if all_lines[i].strip() == "Payment Due Date":
        possible_date = all_lines[i + 1].strip()
        try:
            # Try parsing the date in common formats, e.g., 25 July 2025, 25/07/2025, etc.
            parsed = datetime.strptime(possible_date, "%d %B %Y")  # Example format: 25 July 2025
            due_date = parsed.strftime("%d-%b-%y")  # Format: DD-Mon-YY
        except ValueError:
            # If parsing fails, assign None
            due_date = None
        break

# Print the final due date
print(f"Payment Due Date: {due_date}")

Payment Due Date: 28-Jul-25


In [76]:
### Generate ordered list of transactions in TSV and Filtering out payment transactions




i = 0
last_month_num = None
#year_tracker = current_year        # Will be used later
#year_shifted = False               # Will be used later

while i < len(all_lines) - 3:
    item = all_lines[i].strip()

    if date_pattern.match(item):
        date_str = item
        desc = all_lines[i + 1].strip()
        amount = all_lines[i + 2].strip()
        ref = all_lines[i + 3].strip()

        # Debug
        # print(f"Date, Desc, Amount, Ref : {date_str}\t{desc}\t{ref}\t{amount}")

        # Filter out Bpay Payments
        if payment_ref_pattern.match(ref) and desc == payment_description and negative_amount_pattern.match(amount):
            # Debug
            #print(f"Payment Transaction (skipped) : {date_str}\t{desc}\t{ref}\t{amount}")
            i += 4
            continue

        if ref_pattern.match(ref) and amount_pattern.match(amount):
            # Extract month and day
            try:
                mon, day = date_str.split()
                day = int(day)
                mon_num = month_order.get(mon)
                # Debug
                #print(f"mon  day  mon_num : {mon}\t{day}\t{mon_num}")

                # Adjust year based on month boundary logic
                #if not year_shifted and last_month_num is not None and mon_num > last_month_num:
                #    year_tracker -= 1  # Crossed Dec->Jan boundary, shift to previous year
                #    year_shifted = True

                last_month_num = mon_num

                # Create full date object for sorting
                #full_date = datetime(year_tracker, mon_num, day)       # Will be used later
                full_date = datetime(current_year, mon_num, day)

                # Append with sortable date object
                tsv_with_dates.append((full_date, f"{date_str}\t{desc}\t{amount}"))
                i += 4
            except:
                i += 1  # Skip malformed entries
        else:
            i += 1
    else:
        i += 1

# Sort by actual datetime
tsv_with_dates.sort(key=lambda x: x[0])


# Build final TSV lines with "Mon DD, YYYY  <desc>\t<amount>" format
for _, line in tsv_with_dates:
    parts = line.split("\t")
    if len(parts) == 3:
        date_str, desc, amount = parts
        date_with_year = f"{date_str}, {current_year}"
        combined = f"{date_with_year}  {desc}\t{amount}"
        tsv_lines.append(combined)



In [77]:
### Debugging purpose : output line number.

print("all_lines => ", len(all_lines))
print("All transactions tab separated lines => ", len(tsv_lines))





all_lines =>  544
All transactions tab separated lines =>  87


In [82]:
# Final output : Print summary, and all tab-separated lines
print(f"Lines {len(tsv_lines)}")
print(f"\n{closing_balance}\t{due_date}")
print(f"Closing balance {closing_balance}")
print(f"Payment Due Date: {due_date}" + "\n" * 4)
print("\n".join(tsv_lines))


Lines 87

$5,945.46	28-Jul-25
Closing balance $5,945.46
Payment Due Date: 28-Jul-25




Jun 11, 2025  Stan.Com.Au Sydney Au	22.00
Jun 12, 2025  India At Home Fountain Gateau	5.36
Jun 15, 2025  Mks Spices N Things Da Dandenong Au	34.35
Jun 15, 2025  Budget Grocery Berwick Au	31.51
Jun 15, 2025  Mks Spices`N Things Dandenong Au	4.95
Jun 15, 2025  Mks Spices`N Things Dandenong Au	9.03
Jun 15, 2025  Eagle Dingley Dingley Villaau	82.32
Jun 15, 2025  Bunnings 748000 Narre Warren Au	19.50
Jun 16, 2025  Berwick Berwick Au	23.90
Jun 16, 2025  Dbs*Gkr R5 Port Adelaideau	34.50
Jun 18, 2025  Post Berwick Post Shop Berwick Au	9.15
Jun 18, 2025  Compasspay Thornbury Au	-90.00
Jun 19, 2025  Woolworths 3167 Berwick Au	97.00
Jun 20, 2025  Budget Grocery Berwick Au	17.08
Jun 20, 2025  Coles 0544 Berwick Au	8.25
Jun 20, 2025  Liquorland 3776 Berwick Au	108.00
Jun 20, 2025  Disney Plus 1800-965160 Au	20.99
Jun 21, 2025  Alira Food Iga Berwick Au	73.00
Jun 21, 2025  The Trustee For Dap Tr Heatherton Au	210

In [ ]:
# TODO 
print(f"###################\n")
print(f"Optimzed the code so all : balance, due date, and list of tsv can be generated in single iteration\n")
print(f"###################\n")


###################

Optimzed the code, so all balance, due date, and list of tsv can be generated in single iteration

###################

